In [ ]:
import pandas as pd

from geoscore_de.data_flow.migration import load_migration_data

df = pd.read_csv(
    "../data/raw/migration.csv",
    sep=";",
    encoding="latin1",
    skiprows=4,
    skipfooter=4,
    na_values=["-", "."],
)
df.rename(
    columns={
        "Unnamed: 0": "Year",
        "Unnamed: 1": "MU_ID",
        "Unnamed: 2": "Municipality",
        "Zuzüge über die Gemeindegrenzen": "in_migration",
        "Fortzüge über die Gemeindegrenzen": "out_migration",
    },
    inplace=True,
)

df = load_migration_data("../data/raw/migration.csv")

In [ ]:
df

In [ ]:
import numpy as np

df["in_out_ratio"] = df["in_migration"] / df["out_migration"]
df["log_in_out_ratio"] = df["in_out_ratio"].apply(lambda x: np.log(x) if pd.notnull(x) else x)

In [ ]:
import plotnine as gg

# show rate of in/out migration as histogram
(gg.ggplot(df) + gg.aes(x="in_out_ratio") + gg.geom_histogram() + gg.labs(title="In/Out Migration Ratio Distribution"))

In [ ]:
# now show it on map of germany
import geopandas as gpd

gdf = gpd.read_file("../data/gemeinden_simplify200.geojson")
gdf["ID"] = gdf["AGS"].astype("str")
df["MU_ID"] = df["MU_ID"].astype("str")

In [ ]:
gdf_merged = gdf.merge(df, left_on="ID", right_on="AGS", how="left", indicator=True)

In [ ]:
# print count of missing values
gdf_merged["_merge"].value_counts()

In [ ]:
import matplotlib.pyplot as plt

# use colormap that center is 1 (in_out_ratio = 1 means equal in and out migration)
gdf_merged.plot(
    column="in_out_ratio",
    legend=True,
    cmap="coolwarm",
    figsize=(10, 10),
    vmin=0,
    vmax=2,
    missing_kwds={"color": "black"},
)
plt.title("In/Out Migration Ratio in German Municipalities")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# use colormap that center is 1 (in_out_ratio = 1 means equal in and out migration)
gdf_merged.plot(
    column="log_in_out_ratio",
    legend=True,
    cmap="coolwarm",
    figsize=(10, 10),
    vmin=-1,
    vmax=1,
    missing_kwds={"color": "black"},
)
plt.title("Log In/Out Migration Ratio in German Municipalities")
plt.show()